In [ ]:
################################################################################
# Author 1:      Firstname Lastname
# MatNr 1:       01234567
# Author 2:      Firstname Lastname
# MatNr 2:       01234567
# Author 3:      Jakob Marktl
# MatNr 3:       12335939
# File:          assignment2.ipynb
# Description: ... short description of the file ...
# Comments:    ... comments for the tutors ...
#              ... can be multiline ...
################################################################################

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import pycountry
import pycountry_convert

Alles in einem File, programmieren

Robert:

David:

In [1]:
print("David hat was gemacht")

David hat was gemacht


Jakob:

In [22]:
FishBoats = pd.read_csv('Fish_Boote.csv')
FishInland = pd.read_csv('Fish_Inland.csv')

# FISH BOATS
# Alphabetisch sortiert
FishBoatsSorted = FishBoats.sort_values(by='Reference area', ascending=True)

# transfering mentionable values into lists
boat_referenceArea = FishBoatsSorted['Reference area'].tolist()
boat_timePeriod = FishBoatsSorted['TIME_PERIOD'].tolist()
boat_obsValue = FishBoatsSorted['OBS_VALUE'].tolist()

boat_Data = pd.DataFrame({
    'country': boat_referenceArea,
    'absolute_value': boat_obsValue,
    'timePeriod': boat_timePeriod
})
# FISH INLAND
# sort
FishInlandsorted = FishInland.sort_values(by='Reference area', ascending=True)

# Transfering mentionable values into lists
inland_referenceArea = FishInlandsorted['Reference area'].tolist()
inland_timePeriod = FishInlandsorted['TIME_PERIOD'].tolist()
inland_obsValue = FishInlandsorted['OBS_VALUE'].tolist()

inland_Data = pd.DataFrame({
    'country': inland_referenceArea,
    'absolute_value': inland_obsValue,
    'timePeriod': inland_timePeriod
})

# Function for Bar Chart
def create_casual_chart(data, title, year_filter):

    # set Filter to 2010
    filtered_Data = data[data['timePeriod'] == year_filter]

    # Scatter Plot using plotly.express
    fig = px.bar(filtered_Data, x='country', y='absolute_value', 
                    title=title,
                    labels={'Time_Period': 'Time Period (Years)', 'absolute_value': 'Amount of vessels'})
    # logahritmic scale:
    fig.update_yaxes(type='log')
    fig.update_layout(
        xaxis=dict(tickangle=45)  # Rotates x-axis labels by 45 degrees
    )
    # Show the plot
    fig.show()

# Create chart for Boats
create_casual_chart(boat_Data,"Scatter Plot of amount of fishing vessels in each Country for 2010",2010)
# Create chart for catches
create_casual_chart(inland_Data,"Scatter Plot of amount of fishing vessels in each Country for 2010",2010)




# Mixed Scatter Plot

# finding common countries:
common_countries = set(boat_Data['country']).intersection(inland_Data['country'])
boat_Data_filtered = boat_Data[boat_Data["country"].isin(common_countries)]
inland_Data_filtered = inland_Data[inland_Data["country"].isin(common_countries)]
# finding common time period
common_time = set(boat_Data['timePeriod']).intersection(inland_Data['timePeriod'])
boat_Data_filtered = boat_Data[boat_Data["timePeriod"].isin(common_time)]
inland_Data_filtered = inland_Data[inland_Data["timePeriod"].isin(common_time)]

print(boat_Data_filtered,inland_Data_filtered)




       country  absolute_value  timePeriod
0    Argentina           925.0        2016
1    Argentina           859.0        2020
2    Argentina           804.0        2017
3    Argentina           938.0        2015
4    Argentina           911.0        2014
..         ...             ...         ...
410   Viet Nam        109356.0        2015
411   Viet Nam        128538.0        2010
412   Viet Nam        127784.0        2011
413   Viet Nam        118789.0        2013
414   Viet Nam        112939.0        2014

[415 rows x 3 columns]        country  absolute_value  timePeriod
0    Argentina        14580.00        2014
1    Argentina        18298.00        2011
2    Argentina        19432.85        2020
3    Argentina        15445.00        2010
4    Argentina        14323.00        2012
..         ...             ...         ...
642   Viet Nam       146416.07        2019
643   Viet Nam       148133.10        2020
644   Viet Nam       149504.09        2021
645   Viet Nam       167252.00